In [1]:
import imb_modeling
import pandas as pd
import numpy as np
import time

from sklearn import svm, datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, roc_auc_score
import joblib

from xgboost import XGBClassifier

Load dataset

In [2]:
svd8 = pd.read_csv('data/svd_8.csv',index_col=0)
svd_z5 = pd.read_csv("data/svd_z_5.csv",index_col=0)
svd_z9 = pd.read_csv("data/svd_z_9.csv",index_col=0)

In [3]:
svd8.shape, svd_z5.shape, svd_z9.shape

((6819, 9), (6819, 6), (6819, 10))

In [4]:
svd8.head()

,axis1,axis2,axis3,axis4,axis5,axis6,axis7,axis8,Bankrupt
0,5.995932e+09,2.565263e+09,1.998551e+09,-1.556894e+09,-2.872628e+09,-1.131120e+09,-6.199528e+08,-3.087006e+09,1
1,1.046412e+10,-2.025294e+09,7.646973e+09,2.906900e+09,-1.599878e+09,-1.508080e+09,-2.054188e+08,-2.352258e+09,1
2,5.855594e+09,-2.207877e+09,-2.404231e+09,-1.810148e+09,-2.697782e+09,-1.499563e+09,1.818762e+09,-3.908349e+08,1
3,1.071762e+10,1.341745e+09,6.819712e+09,7.430153e+08,-4.009937e+09,2.112051e+09,7.794637e+09,4.507580e+09,1
4,9.244816e+09,7.462440e+09,-1.875104e+09,3.030693e+09,-2.042418e+09,-3.254255e+08,-3.596034e+08,2.267177e+09,1


Params

In [14]:
# SVM parameter dictionary
SVM_params = {'kernel':['rbf'],  #linear 
              'C':[1, 10]}

# Parameters for Decision treee
dt_params = {'criterion':['gini','entropy'], 
              'max_depth' : range(1,10), 
              'min_samples_split' : range(1,10), 
              'min_samples_leaf': range(1,5)}

# Parameters for Random Forest
rf_params = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 3)], # Number of trees in random forest
               'max_features': ['auto', 'sqrt'], # Number of features to consider at every split
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 3)], # Maximum number of levels in tree
               'min_samples_split': [2, 5, 10], # Minimum number of samples required to split a node
               'min_samples_leaf': [1, 2, 4], # Minimum number of samples required at each leaf node
               'bootstrap': [True, False] # Method of selecting samples for training each tree
                }

#Parameters for gradient boosting
gb_params = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 3),
    "min_samples_leaf": np.linspace(0.1, 0.5, 3),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.4, 0.7, 1.0],
    "n_estimators":[10]
    }

#Parameters for XGBoost
xgb_estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    tree_method='gpu_hist',
    seed=421
)
xgb_params = {
    'max_depth': range(2, 10, 2),
    'n_estimators': range(60, 220, 20),
    "min_child_weight":[1, 3, 5],
    'learning_rate': [0.1, 0.01, 0.05]
}

In [6]:
models = [(svm.SVC(),SVM_params),
          (DecisionTreeClassifier(),dt_params),
          (RandomForestClassifier(),rf_params),
          (GradientBoostingClassifier(),gb_params),
          (xgb_estimator,xgb_params)]

In [7]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall':make_scorer(recall_score),
           'roc_auc':make_scorer(roc_auc_score)}

Modeling

In [15]:
res = []
for model, param in models[2:3]:
    start = time.time()
    print('Training: ' + str(model))
    key_modified_params = {f"model__{key}": val for key, val in param.items()}
    output = imb_modeling.imb_pipe_fit(model,key_modified_params,X=svd8.drop(columns='Bankrupt'),y=svd8['Bankrupt'])
    res.append(output)
    end = time.time()
    print(output['cv_score'])
    print(output['test_score'])
    print(output['best_param'])
    
    print('Time: ' + str(end-start))

Training: RandomForestClassifier()
fitting model...
Fitting 2 folds for each of 3960 candidates, totalling 7920 fits


/Users/nicole/miniconda3/envs/msia420-2/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/nicole/miniconda3/envs/msia420-2/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/nicole/miniconda3/envs/msia420-2/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_

[CV 1/2] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=200;, score=0.714 total time=   1.7s
[CV 2/2] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=200;, score=0.653 total time=   1.8s


/Users/nicole/miniconda3/envs/msia420-2/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/nicole/miniconda3/envs/msia420-2/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 2/2] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=400;, score=0.650 total time=   3.8s
[CV 1/2] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=400;, score=0.719 total time=   3.9s


...

[CV 1/2] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=4, model__min_samples_split=10, model__n_estimators=600;, score=0.718 total time=   6.5s
[CV 2/2] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=4, model__min_samples_split=5, model__n_estimators=1800;, score=0.655 total time=  17.6s


/Users/nicole/miniconda3/envs/msia420-2/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/nicole/miniconda3/envs/msia420-2/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 1/2] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=4, model__min_samples_split=5, model__n_estimators=1800;, score=0.716 total time=  18.3s


/Users/nicole/miniconda3/envs/msia420-2/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/nicole/miniconda3/envs/msia420-2/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/nicole/miniconda3/envs/msia420-2/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_

KeyboardInterrupt: 

In [19]:
range(1,10)

range(1, 10)